In [9]:
import pandas as pd
import numpy as np
import re

# 2. 전처리

In [10]:
origin_data=pd.read_csv("전처리파일저장/cgv_원본_통합파일.csv", encoding='utf-8-sig')
data=origin_data.copy()

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화제목     8193 non-null   object 
 1   에그지수     6400 non-null   float64
 2   성별       5395 non-null   object 
 3   연령별      5576 non-null   object 
 4   감독/등장인물  7001 non-null   object 
 5   관람포인트    7444 non-null   object 
 6   실관람평개수   8193 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 448.2+ KB


In [12]:
data.head()

,영화제목,에그지수,성별,연령별,감독/등장인물,관람포인트,실관람평개수
0,매트릭스3 레볼루션,0.0,NaN,NaN,"감독 앤디 워쇼스키, 감독 라나 워쇼스키, 키아누 리브스, 모니카 벨루치, 로렌스 피쉬번",영상미가 뛰어나고 몰입하여 보게 되는 영화,0
1,써클,NaN,NaN,NaN,"감독 박기형, 김혜성, 이행석, 이태성, 장희진",배우연기가 메소드급인 몰입하여 보게 되는 영화,0
2,폴리와 함께,0.0,NaN,NaN,"감독 존 햄버거, 필립 세이무어 호프만, 데브라 메싱, 제니퍼 애니스톤, 알렉 볼드윈",배우연기가 메소드급인 몰입하여 보게 되는 영화,0
3,온화한 일상,NaN,NaN,NaN,NaN,스토리가 탄탄하고 공감되는 영화,5
4,시월애,95.0,"성별예매 분포: 남 45%, 여 55%","연령별예매 분포: 10대 2%, 20대 31%, 30대 40%, 40대 14%, 5...","감독 이현승, 이인철, 전지현, 이정재, 김무생",영상미가 뛰어나고 감동적인 영화,238


# 2주차 데이터 까지 사용
즉, 평점과 같이 2주차 이후에나 얻을 수 있는 데이터는 모두 제거한다

In [13]:
data=data[["영화제목", "감독/등장인물"]]
data.head()

,영화제목,감독/등장인물
0,매트릭스3 레볼루션,"감독 앤디 워쇼스키, 감독 라나 워쇼스키, 키아누 리브스, 모니카 벨루치, 로렌스 피쉬번"
1,써클,"감독 박기형, 김혜성, 이행석, 이태성, 장희진"
2,폴리와 함께,"감독 존 햄버거, 필립 세이무어 호프만, 데브라 메싱, 제니퍼 애니스톤, 알렉 볼드윈"
3,온화한 일상,NaN
4,시월애,"감독 이현승, 이인철, 전지현, 이정재, 김무생"


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화제목     8193 non-null   object
 1   감독/등장인물  7001 non-null   object
dtypes: object(2)
memory usage: 128.1+ KB


In [15]:
data.isna().sum()

영화제목          0
감독/등장인물    1192
dtype: int64

## 감독/등장인물

In [16]:
data.dropna(subset=['감독/등장인물'], inplace=True)

In [17]:
people=data[["감독/등장인물"]]

# 2. 괄호를 고려하여 문자열을 분리하고, 감독과 배우를 추출하는 핵심 함수
def parse_roles(text: str) -> tuple[list[str], list[str]]:
    """
    하나의 문자열을 입력받아 감독 리스트와 배우 리스트를 반환합니다.
    괄호 안의 쉼표는 분리 기준으로 삼지 않아 안정적으로 처리합니다.
    """
    
    # 괄호 안의 쉼표(,)를 임시 문자열('##COMMA##')로 치환하는 내부 함수
    def replace_comma_in_parentheses(match: re.Match) -> str:
        return match.group(0).replace(',', '##COMMA##')
    
    # 정규표현식을 사용해 괄호(...) 안의 내용에만 위의 치환 함수를 적용
    temp_text = re.sub(r'\([^)]+\)', replace_comma_in_parentheses, text)
    
    # 이제 쉼표로 분리해도 괄호 안 내용은 분리되지 않음
    # 분리 후, 임시 문자열을 다시 쉼표로 복원하고 양쪽 공백을 제거
    people_list = [p.replace('##COMMA##', ',').strip() for p in temp_text.split(',')]
    
    directors = []
    actors = []
    
    for person in people_list:
        if person.startswith('감독 '):
            # '감독 ' 접두사를 제거하고 감독 리스트에 추가
            directors.append(person.replace('감독 ', '', 1))
        elif person:  # person이 빈 문자열이 아닌 경우
            # 배우 리스트에 추가
            actors.append(person)
            
    return directors, actors

people[['감독리스트', '배우리스트']] = people["감독/등장인물"].apply(
    lambda text: pd.Series(parse_roles(text))
)


# 4. 감독 리스트를 개별 컬럼('감독1', '감독2', ...)으로 분리
# 리스트의 최대 길이를 구해 필요한 만큼 컬럼을 동적으로 생성합니다.
max_directors = people['감독리스트'].str.len().max()
for i in range(1, max_directors + 1):
    # .str.get(i-1)을 사용해 각 리스트의 i번째 요소를 안전하게 추출 (없으면 NaN)
    people[f'감독{i}'] = people['감독리스트'].str.get(i - 1)

# 5. 배우 리스트를 개별 컬럼('배우1', '배우2', ...)으로 분리
max_actors = people['배우리스트'].str.len().max()
for i in range(1, max_actors + 1):
    people[f'배우{i}'] = people['배우리스트'].str.get(i - 1)

# 6. 최종 결과 확인
# 원본 및 중간 과정 컬럼을 제외하고, 최종적으로 생성된 감독/배우 컬럼만 선택하여 출력합니다.
final_columns = [col for col in people.columns if re.match(r'^(감독\d+|배우\d+)$', col)]
people = people[final_columns]
people

,감독1,감독2,감독3,감독4,감독5,배우1,배우2,배우3,배우4,배우5,배우6
0,앤디 워쇼스키,라나 워쇼스키,NaN,NaN,NaN,키아누 리브스,모니카 벨루치,로렌스 피쉬번,NaN,NaN,NaN
1,박기형,NaN,NaN,NaN,NaN,김혜성,이행석,이태성,장희진,NaN,NaN
2,존 햄버거,NaN,NaN,NaN,NaN,필립 세이무어 호프만,데브라 메싱,제니퍼 애니스톤,알렉 볼드윈,NaN,NaN
4,이현승,NaN,NaN,NaN,NaN,이인철,전지현,이정재,김무생,NaN,NaN
5,켄 로치,NaN,NaN,NaN,NaN,킬리언 머피,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8188,구스 반 산트,NaN,NaN,NaN,NaN,맷 데이먼,로즈마리 드윗,존 크래신스키,프란시스 맥도맨드,NaN,NaN
8189,김태윤(1),NaN,NaN,NaN,NaN,김수로,오광록,이선균,고은아,NaN,NaN
8190,잭 스나이더,NaN,NaN,NaN,NaN,샘 닐,짐 스터게스,헬렌 미렌,제프리 러쉬,NaN,NaN
8191,야노 히로유키,오가 슌지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
data.drop(columns=['감독/등장인물'], inplace=True)
data=pd.concat([data, people], axis=1)
data.head()

,영화제목,감독1,감독2,감독3,감독4,감독5,배우1,배우2,배우3,배우4,배우5,배우6
0,매트릭스3 레볼루션,앤디 워쇼스키,라나 워쇼스키,NaN,NaN,NaN,키아누 리브스,모니카 벨루치,로렌스 피쉬번,NaN,NaN,NaN
1,써클,박기형,NaN,NaN,NaN,NaN,김혜성,이행석,이태성,장희진,NaN,NaN
2,폴리와 함께,존 햄버거,NaN,NaN,NaN,NaN,필립 세이무어 호프만,데브라 메싱,제니퍼 애니스톤,알렉 볼드윈,NaN,NaN
4,시월애,이현승,NaN,NaN,NaN,NaN,이인철,전지현,이정재,김무생,NaN,NaN
5,보리밭을 흔드는 바람,켄 로치,NaN,NaN,NaN,NaN,킬리언 머피,NaN,NaN,NaN,NaN,NaN


# 저장

In [19]:
data.to_csv("전처리파일저장/cgv_전처리_통합파일.csv", index=False, encoding='utf-8-sig')